# Import requirements

In [1]:
!pip install Sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 14.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 15.3 MB/s 
     |████████████████████████████████| 6.6 MB 72.5 MB/s 
     |████████████████████████████████| 163 kB 83.4 MB/s 


In [ ]:
import os
import pdb
import argparse
from dataclasses import dataclass, field
from typing import Optional
from collections import defaultdict

import torch
from torch.nn.utils.rnn import pad_sequence

import numpy as np
from tqdm import tqdm, trange

from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    AutoConfig,
    AdamW,
    
    AlbertTokenizer, AlbertForSequenceClassification
)

# 1. Preprocess

In [ ]:
def make_id_file(task, tokenizer):
    def make_data_strings(file_name):
        data_strings = []
        with open(os.path.join(file_name), 'r', encoding='utf-8') as f:
            id_file_data = [tokenizer.encode(line.lower()) for line in f.readlines()]
        for item in id_file_data:
            data_strings.append(' '.join([str(k) for k in item]))
        return data_strings
    
    print('it will take some times...')
    train_pos = make_data_strings('sentiment.train.1')
    train_neg = make_data_strings('sentiment.train.0')
    dev_pos = make_data_strings('sentiment.dev.1')
    dev_neg = make_data_strings('sentiment.dev.0')

    print('make id file finished!')
    return train_pos, train_neg, dev_pos, dev_neg

In [ ]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving sentiment.train.1 to sentiment.train.1
Saving sentiment.train.0 to sentiment.train.0
Saving test_no_label.csv to test_no_label.csv
Saving sentiment.dev.1 to sentiment.dev.1
Saving sentiment.dev.0 to sentiment.dev.0


In [ ]:
!ls

/bin/bash: ls3: command not found


In [ ]:
train_pos, train_neg, dev_pos, dev_neg = make_id_file('yelp', tokenizer)

it will take some times...
make id file finished!


In [ ]:
train_pos[:10]

['2 5977 950 13 9 3',
 '2 18656 7705 365 13 9 3',
 '2 59 67 57 1954 621 18 17 1392 5262 56 25 510 254 13 9 3',
 '2 32 13 22 18 21 254 13865 69 20538 7298 13 9 3',
 '2 14 1138 25 4753 13 9 3',
 '2 254 748 950 13 9 3',
 '2 254 365 13 9 3',
 '2 11554 16 208 25 27269 17 7503 16 621 18 13 9 3',
 '2 374 209 26 4311 54 748 16385 18 17 3911 13 9 3',
 '2 14 78 978 1879 5289 13 9 3']

In [ ]:
class SentimentDataset(object):
    def __init__(self, tokenizer, pos, neg):
        self.tokenizer = tokenizer
        self.data = []
        self.label = []

        for pos_sent in pos:
            self.data += [self._cast_to_int(pos_sent.strip().split())]
            self.label += [[1]]
            # print('pos_sent: ', self.label, pos_sent)
        for neg_sent in neg:
            self.data += [self._cast_to_int(neg_sent.strip().split())]
            self.label += [[0]]

    def _cast_to_int(self, sample):
        return [int(word_id) for word_id in sample]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return np.array(sample), np.array(self.label[index])

In [ ]:
train_dataset = SentimentDataset(tokenizer, train_pos, train_neg)
dev_dataset = SentimentDataset(tokenizer, dev_pos, dev_neg)

In [ ]:
# for i, item in reversed(list(enumerate(train_dataset))):
#     print(item)
#     if i == 1:
#         break

# for i, item in enumerate(train_dataset):
#     print(item)
#     if i == 1:
#         break

In [ ]:
# def collate_fn_style(samples):
#     # print('samples : ',samples)
#     input_ids, labels = zip(*samples)
#     # print('inputs : ', input_ids)
#     # print('labels : ', labels)

#     # batch 사이즈중 가장 긴 문장의 길이 추출
#     max_len = max(len(input_id) for input_id in input_ids)

#     # 조금 더 잘 pad 를 하기 위해 문장 정리
#     sorted_indices = np.argsort([len(input_id) for input_id in input_ids])[::-1]
    
#     # 위에 오류
#     # sorted_indices = range(len(input_ids))

#     # https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html, 길이를 맞추기 위해 pad 를 해줌. 전체적인 dim 이 같아짐.
#     input_ids = pad_sequence([torch.tensor(input_ids[index]) for index in sorted_indices],
#                              batch_first=True)
    
#     # 불필요한 영역에는 attention 을 두지 않도록 attention mask 생성.
#     attention_mask = torch.tensor(
#         [[1] * len(input_ids[index]) + [0] * (max_len - len(input_ids[index])) for index in
#          sorted_indices])

#     # attention_mask = []
#     # for seq in input_ids:
#     #   seq_mask = [float(i>0) for i in seq]
#     #   attention_mask.append(seq_mask)
    
#     # attention_mask = torch.tensor(attention_mask)
      
#     token_type_ids = torch.tensor([[0] * len(input_ids[index]) for index in sorted_indices])
#     position_ids = torch.tensor([list(range(len(input_ids[index]))) for index  in sorted_indices])
#     labels = torch.tensor(np.stack(labels, axis=0)[sorted_indices])

#     return input_ids, attention_mask, token_type_ids, position_ids, labels


def collate_fn_style(samples):
    input_ids, labels = zip(*samples)
    max_len = max(len(input_id) for input_id in input_ids)
    # sorted_indices = np.argsort([len(input_id) for input_id in input_ids])[::-1] 
    sorted_indices = range(len(input_ids))

    # https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html,
    # 길이를 맞추기 위해 pad 를 해줌. batch 마다 적용
    input_ids = pad_sequence([torch.tensor(input_ids[index]) for index in sorted_indices],
                             batch_first=True)
    # attention_mask = torch.tensor(
    #     [[1] * len(input_ids[index]) + [0] * (max_len - len(input_ids[index])) for index in
    #      sorted_indices])

    attention_mask = []
    for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_mask.append(seq_mask)
    attention_mask = torch.tensor(attention_mask)

    token_type_ids = torch.tensor([[0] * len(input_ids[index]) for index in sorted_indices])
    position_ids = torch.tensor([list(range(len(input_ids[index]))) for index in sorted_indices])
    labels = torch.tensor(np.stack(labels, axis=0)[sorted_indices])
    

    return input_ids, attention_mask, token_type_ids, position_ids, labels

In [ ]:
train_batch_size=128 # 32-> 128
eval_batch_size=256 # 64-> 256

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=train_batch_size,
                                           shuffle=True, collate_fn=collate_fn_style,
                                           pin_memory=True, num_workers=2)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=eval_batch_size,
                                         shuffle=False, collate_fn=collate_fn_style,
                                         num_workers=2)

In [ ]:
# for idx, i in enumerate(train_loader):
#     input_ids, attention_mask, token_type_ids, position_ids, labels = i
#     print(i)
#     if idx == 0:
#       break

[tensor([[   2,   48,  209,  ...,    0,    0,    0],
        [   2, 2170,   14,  ...,    0,    0,    0],
        [   2,   13, 1373,  ...,    0,    0,    0],
        ...,
        [   2,   31, 1905,  ...,    0,    0,    0],
        [   2, 1138,   23,  ...,    0,    0,    0],
        [   2,  207,   13,  ...,    0,    0,    0]]), tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.]]), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), tensor([[ 0,  1,  2,  ..., 19, 20, 21],
        [ 0,  1,  2,  ..., 19, 20, 21],
        [ 0,  1,  2,  ..., 19, 20, 21],
        ...,
        [ 0,  1,  2,  ..., 19, 20, 21],
        [ 0,  1,  2,  ...

In [ ]:
# random seed
random_seed=33 # 42 -> 33
np.random.seed(random_seed)
torch.manual_seed(random_seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = AlbertForSequenceClassification.from_pretrained('albert-base-v2')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# GPU 에 얹어주는 작업
model.to(device)

Downloading:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.decoder.weight', 'predictions.bias', 'predictions.decoder.bias', 'predictions.dense.bias', 'predictions.dense.weight', 'predictions.LayerNorm.bias', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

In [ ]:
model.train()
learning_rate = 5e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
def compute_acc(predictions, target_labels):
    return (np.array(predictions) == np.array(target_labels)).mean()

In [ ]:
train_epoch = 3
lowest_valid_loss = 9999.

# early stopping
patience_limit = 2
patience_check = 0


for epoch in range(train_epoch):

    # tqdm 은 Bar 형태로 학습량을 알려주는 module.
    with tqdm(train_loader, unit="batch") as tepoch:


        for iteration, (input_ids, attention_mask, token_type_ids, position_ids, labels) in enumerate(tepoch):
            tepoch.set_description(f"Epoch {epoch}")

            # 모든 데이터를 GPU 로 옮겨줌.
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            position_ids = position_ids.to(device)
            labels = labels.to(device, dtype=torch.long)

            optimizer.zero_grad()

            # forward-propagation 연산 진행
            output = model(input_ids=input_ids,
                           attention_mask=attention_mask,
                           token_type_ids=token_type_ids,
                           position_ids=position_ids,
                           labels=labels)

            loss = output.loss

            # back-propagation 연산
            loss.backward()

            # 가중치 update
            optimizer.step()  

            # tqdm 에서 loss 를 출력하기 위해 loss 를 넣어줌.
            tepoch.set_postfix(loss=loss.item())


            # int(len(train_loader) / 5) == 0 마다 validation set 으로 evaluate.
            if iteration != 0 and iteration % int(len(train_loader) / 5) == 0:
                # Evaluate the model five times per epoch
                with torch.no_grad():
                    model.eval()
                    valid_losses = []
                    predictions = []
                    target_labels = []
                    for input_ids, attention_mask, token_type_ids, position_ids, labels in tqdm(dev_loader,
                                                                                                desc='Eval',
                                                                                                position=1,
                                                                                                leave=None):
                        input_ids = input_ids.to(device)
                        attention_mask = attention_mask.to(device)
                        token_type_ids = token_type_ids.to(device)
                        position_ids = position_ids.to(device)
                        labels = labels.to(device, dtype=torch.long)

                        output = model(input_ids=input_ids,
                                       attention_mask=attention_mask,
                                       token_type_ids=token_type_ids,
                                       position_ids=position_ids,
                                       labels=labels)

                        # logits : tensor([[ 2.0392, -1.4302], [-1.1802,  2.7419], ...])
                        logits = output.logits

                        # loss :  tensor(0.1518, device='cuda:0', grad_fn=<NllLossBackward0>)
                        loss = output.loss
                        valid_losses.append(loss.item())

                        batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
                        batch_labels = [int(example) for example in labels]

                        predictions += batch_predictions
                        target_labels += batch_labels


                # valid_losses : [0.07377569377422333, 0.0990561693906784, ..., 0.02395477145910263], len == 16
                # predictions :  [0, 0, 0, ..., 0] , len == 4000
                # target_labels : [1, 1, 1, ..., 0] , len == 4000
                # np.array(predictions) == np.array(target_labels) : [ True ...  True  True  True]
                # (np.array(predictions) == np.array(target_labels)).mean() : 0.9705

                acc = compute_acc(predictions, target_labels)
                valid_loss = sum(valid_losses) / len(valid_losses)
                if lowest_valid_loss > valid_loss:
                    lowest_valid_loss = valid_loss
                    print('Acc for model which have lower valid loss: ', acc, ', current loss is', valid_loss, ', lowest_valid_loss :', lowest_valid_loss)
                    torch.save(model.state_dict(), "./pytorch_model.bin")

                # early stopping 코드 추가
                    patience_check = 0
                else:
                    print('Lower than previous accuracy, accuracy is : ', acc, ', current loss is', valid_loss)
                    patience_limit += 1
                    if patience_check >= patience_limit: # early stopping 조건 만족 시 조기 종료
                        print('Ended training for bigger patience_check than limit')
                        break


Epoch 0:  20%|██        | 693/3463 [02:00<41:23,  1.12batch/s, loss=0.0767]

Acc for model which have lower valid loss:  0.9775 , current loss is 0.06242226355243474 , lowest_valid_loss : 0.06242226355243474


Epoch 0:  40%|████      | 1386/3463 [04:00<22:56,  1.51batch/s, loss=0.0361]

Lower than previous accuracy, accuracy is :  0.977 , current loss is 0.06658012693515047


Epoch 0:  60%|█████▉    | 2077/3463 [05:59<19:40,  1.17batch/s, loss=0.0719]

...

Epoch 2:  80%|███████▉  | 2770/3463 [07:58<07:29,  1.54batch/s, loss=0.0543]

Lower than previous accuracy, accuracy is :  0.97825 , current loss is 0.060896553099155426


Epoch 2: 100%|█████████▉| 3462/3463 [09:58<00:00,  1.51batch/s, loss=0.00831]

Lower than previous accuracy, accuracy is :  0.97725 , current loss is 0.062197313411161304


Epoch 2: 100%|██████████| 3463/3463 [09:58<00:00,  5.79batch/s, loss=0.0509]


In [ ]:
import pandas as pd
test_df = pd.read_csv('test_no_label.csv')

In [ ]:
test_dataset = test_df['Id']

In [ ]:
def make_id_file_test(tokenizer, test_dataset):
    data_strings = []
    id_file_data = [tokenizer.encode(sent.lower()) for sent in test_dataset]
    for item in id_file_data:
        data_strings.append(' '.join([str(k) for k in item]))
    return data_strings

In [ ]:
test = make_id_file_test(tokenizer, test_dataset)

In [ ]:
test[:10]

['2 32 13 22 18 21 979 78 1496 17 78 11974 18 206 85 42 162 13 9 3',
 '2 86 17841 37 40 315 3839 13 9 3',
 '2 32 25 14 127 14351 7804 19 14 1152 13 9 3',
 '2 107 52 1676 21 9140 29 158 148 13 9 3',
 '2 31 23 4741 17 39 117 55 583 86 5733 17 5575 13 9 3',
 '2 14 104 584 95 420 30 23 1047 23 14 2364 13 9 3',
 '2 90 13 15 52 14 53 18 35 28 291 13 15 14 53 18 19 1630 183 13 9 3',
 '2 59 1499 32 70 431 26 42 17 50 253 15600 13 9 3',
 '2 3123 14 13533 144 13 103 22 38 166 143 184 20 170 14 1428 13 9 3',
 '2 59 57 40 5977 3155 16 22621 18 20 3538 37 13 9 3']

In [ ]:
class SentimentTestDataset(object):
    def __init__(self, tokenizer, test):
        self.tokenizer = tokenizer
        self.data = []

        for sent in test:
            self.data += [self._cast_to_int(sent.strip().split())]

    def _cast_to_int(self, sample):
        return [int(word_id) for word_id in sample]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return np.array(sample)

In [ ]:
test_dataset = SentimentTestDataset(tokenizer, test)

In [ ]:
def collate_fn_style_test(samples):
    input_ids = samples
    max_len = max(len(input_id) for input_id in input_ids)

    # 버그.
    # smaples 가 input id 만 가지고 있음. trainset 은 input_ids 와 labels 둘 다 가졌음. test_dataset 은 labels 이 없음.
    # input 의 순서를 바꿔주면 kaggle 에 내가 모르는 labels set 과 순서가 얻 바뀌게 된다.
    # sorted_indices = np.argsort([len(input_id) for input_id in input_ids])[::-1]
    sorted_indices = range(len(input_ids))

    input_ids = pad_sequence([torch.tensor(input_ids[index]) for index in sorted_indices],
                             batch_first=True)
    # attention_mask = torch.tensor(
    #     [[1] * len(input_ids[index]) + [0] * (max_len - len(input_ids[index])) for index in
    #      sorted_indices])

    attention_mask = []
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_mask.append(seq_mask)
    attention_mask = torch.tensor(attention_mask)

    token_type_ids = torch.tensor([[0] * len(input_ids[index]) for index in sorted_indices])
    position_ids = torch.tensor([list(range(len(input_ids[index]))) for index in sorted_indices])

    return input_ids, attention_mask, token_type_ids, position_ids

In [ ]:
test_batch_size = 32
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size,
                                          shuffle=False, collate_fn=collate_fn_style_test,
                                          num_workers=2)

In [ ]:
with torch.no_grad():
    model.eval()
    predictions = []
    for input_ids, attention_mask, token_type_ids, position_ids in tqdm(test_loader,
                                                                        desc='Test',
                                                                        position=1,
                                                                        leave=None):

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        position_ids = position_ids.to(device)

        output = model(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids,
                       position_ids=position_ids)

        logits = output.logits
        batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
        predictions += batch_predictions


Test:  88%|████████▊ | 28/32 [00:00<00:00, 46.60it/s]
                                                     

In [ ]:
test_df['Category'] = predictions

In [ ]:
test_df.to_csv('submission2.csv', index=False)

# 새 섹션